In [1]:
from Transbot_Lib import Transbot
from ipywidgets import interact
import ipywidgets as widgets
import time
import math

bot = Transbot()

Serial Opened! Baudrate=115200


In [2]:
# 데이터 수신 설정
bot.create_receive_threading()
bot.set_auto_report_state(True, forever=False)

# 조명 설정
bot.set_colorful_effect(5, speed=255, parm=255)

# bot.get_imu_state()

----------------create receive threading--------------


In [ ]:
# 가속도 데이터 확인
try:
    while True:
        a_x, a_y, a_z = bot.get_accelerometer_data()
        print("acce:", a_x, a_y, a_z)
        time.sleep(.1)
except KeyboardInterrupt:
    pass

In [ ]:
# 자이로 데이터 확인
try:
    while True:
        g_x, g_y, g_z = bot.get_gyroscope_data()
        print("gyro:", g_x, g_y, g_z)
        time.sleep(.01)
except KeyboardInterrupt:
    pass

In [ ]:
# PID 제어 설정
kp = 0.05
ki = 0.0
kd = 4.0
bot.set_pid_param(kp, ki, kd, forever=False)
kp, ki, kd = bot.get_motion_pid()
print("PID set to:", kp, ki, kd)

In [3]:
def car_motion(line, angular):
    # 모터 부하 방지
    if abs(line) >= 5 or abs(angular) >= 10:
        speed_l = line / 100.0
        speed_a = angular / 100.0
        bot.set_car_motion(speed_l, speed_a)
        return speed_l, speed_a
    else:
        bot.set_car_motion(0, 0)
        return 0, 0

interact(car_motion, line=widgets.IntSlider(min=-45,max=45,step=1,value=0), \
         angular=widgets.IntSlider(min=-200,max=200,step=1,value=0))

interactive(children=(IntSlider(value=0, description='line', max=45, min=-45), IntSlider(value=0, description=…

<function __main__.car_motion(line, angular)>

In [ ]:
enable = True
bot.set_imu_adjust(enable, forever=False)

In [5]:
enable = False
bot.set_imu_adjust(enable, forever=False)

In [6]:
state = bot.get_imu_state()
print("imu state:", state)

imu state: False


In [15]:
def turn_right(degrees):
    target_angle = degrees * (math.pi / 180)
    current_angle = 0
    previous_time = time.time()
    bot.set_car_motion(0,-0.5)

    while abs(current_angle) < abs(target_angle):
        current_time = time.time()
        dt = current_time - previous_time
        previous_time = current_time

        g_x, g_y, g_z = bot.get_gyroscope_data()
        current_angle += g_z * dt

        time.sleep(0.001)
    bot.set_car_motion(0,0)

In [20]:
def turn_right_pid(degrees, kp, ki, kd):
    target_angle = degrees * (math.pi / 180)
    current_angle = 0
    previous_time = time.time()

    integral = 0
    previous_error = target_angle - current_angle

    while abs(current_angle) < abs(target_angle):
        current_time = time.time()
        dt = current_time - previous_time
        previous_time = current_time

        g_x, g_y, g_z = bot.get_gyroscope_data()
        current_angle += g_z * dt

        error = target_angle - current_angle
        integral += error * dt
        derivative = (error - previous_error) / dt
        previous_error = error

        control_signal = kp * error + ki * integral + kd * derivative
        control_signal = max(min(control_signal, 3), -3)

        bot.set_motor_speed(0, -control_signal)

        time.sleep(0.01)
    bot.set_car_motion(0,0)

In [19]:
turn_right(90)

In [ ]:
turn_right_pid(90)

In [11]:
bot.set_car_motion(0,0)